In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 50.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 3.9MB 51.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=02e61d88214e53f02ba73fdbf846b7084439cd5860ad8f5b7eb92381d9729881
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [2]:
import tensorflow as tf 
tf.__version__

'2.1.0'

In [3]:
import os
#import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU, add, Conv2D, Reshape
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, multiply
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras import utils
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image, text, sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K



import numpy as np

SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)  



In [4]:
from os import listdir, makedirs
from os.path import isfile, join, basename, splitext
from random import seed, shuffle
import glob
import time
# set the matplotlib backend so figures can be saved in the background
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
# import the necessary packages

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.metrics import classification_report

from imutils import paths
import math
import numpy as np
import pickle
import operator
from operator import itemgetter
from itertools import zip_longest
from collections import defaultdict
import json
import joblib
from tqdm import tqdm
import pandas as pd
from nltk.tokenize.treebank import TreebankWordTokenizer
import pandas as pd
import seaborn as sns
import datetime

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%%capture
!unzip /content/drive/My\ Drive/anndl-2020-vqa.zip

In [7]:
from os import listdir, makedirs
from os.path import isfile, join, basename, splitext
from random import seed, shuffle
from PIL import Image
import json
import cv2
from tensorflow.keras.preprocessing.sequence import pad_sequences

imgs_path = '/content/VQA_Dataset/Images'
train_json_path = '/content/VQA_Dataset/train_questions_annotations.json'
test_json_path = '/content/VQA_Dataset/test_questions.json'
  
DATASET_SPLIT = 0.8
BATCH_SIZE = 128
  
classes = {'0': 0,
            '1': 1,
            '2': 2,
            '3': 3,
            '4': 4,
            '5': 5,
            'apple': 6,
            'baseball': 7,
            'bench': 8,
            'bike': 9,
            'bird': 10,
            'black': 11,
            'blanket': 12,
            'blue': 13,
            'bone': 14,
            'book': 15,
            'boy': 16,
            'brown': 17,
            'cat': 18,
            'chair': 19,
            'couch': 20,
            'dog': 21,
            'floor': 22,
            'food': 23,
            'football': 24,
            'girl': 25,
            'grass': 26,
            'gray': 27,
            'green': 28,
            'left': 29,
            'log': 30,
            'man': 31,
            'monkey bars': 32,
            'no': 33,
            'nothing': 34,
            'orange': 35,
            'pie': 36,
            'plant': 37,
            'playing': 38,
            'red': 39,
            'right': 40,
            'rug': 41,
            'sandbox': 42,
            'sitting': 43,
            'sleeping': 44,
            'soccer': 45,
            'squirrel': 46,
            'standing': 47,
            'stool': 48,
            'sunny': 49,
            'table': 50,
            'tree': 51,
            'watermelon': 52,
            'white': 53,
            'wine': 54,
            'woman': 55,
            'yellow': 56,
            'yes': 57}
  
N_CLASSES = len(classes)

In [ ]:
def image_feature_extractor(target_path, image_list, BATCH_SIZE):
	"""
	Extracts (512, 7, 7)-dimensional CNN features and save them locally

	Input:
		target_path: path to save the features
		image_list: image filenames
		BATCH_SIZE: batch size

	Returns:
		None
	"""
	 
	model = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(3, 224, 224)))

  	# add a progress bar
	progbar = utils.Progbar(int(np.ceil(len(image_list) / float(BATCH_SIZE))))

  	# loop over the images in batches
	for (b, i) in enumerate(range(0, len(image_list), BATCH_SIZE)):
		# extract batch of images and prepare them to pass it through the VGG 
		# network for feature extraction

		progbar.update(b+1)
		
		batch_range = range(i, min(i + BATCH_SIZE, len(image_list)))
		batchPaths = image_list[batch_range[0]: batch_range[-1]+1]

		batchImages = []
		batchIds = []
		# loop over the images and labels in the current batch
		for imagePath in batchPaths:

            # load the input image using the Keras helper utility
            # while ensuring the image is resized to 224x224 pixels
			img = image.load_img(os.path.join(imgs_path,imagePath), target_size=(224, 224))
			img = image.img_to_array(img)
    
            # preprocess the image by 
            # (1) expanding the dimensions to include batch dim and
            # (2) subtracting the mean RGB pixel intensity from the ImageNet dataset
			img = np.expand_dims(img, axis=0)
			img = preprocess_input(img)
    
            # add the image to the batch
			batchImages.append(img)
			# image ids of the batch
			batchIds.append(imagePath.split('.')[0][-6:])
	  
		batchImages = np.vstack(batchImages) # (BATCH_SIZE, 3, 224, 224)

		# pass the images through the network and use the outputs as our actual features
		features = model.predict(batchImages) # (BATCH_SIZE, 512, 7, 7)
		features = tf.reshape(features, (features.shape[0], features.shape[1], -1)) # (BATCH_SIZE, 512, 49)
		features = tf.transpose(features, perm =[0,2,1])  # (BATCH_SIZE, 49, 512)

		# loop over the batch to save them locally
		for id, feat in zip(batchIds, features):
			np.save(os.path.join(target_path, id), feat)

In [ ]:
image_list = os.listdir("/content/VQA_Dataset/Images")
BATCH_SIZE = 128
target_path = '/content/drive/My Drive/VQA/features'

In [ ]:
image_feature_extractor(target_path, image_list, BATCH_SIZE)

98/98 [==============================] - 794s 8s/step


In [8]:
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer


def process_sentence(sentence):
    """
    Cleans a given raw sentence
    Input:
        sentence: a raw sentence
    Returns:
        Returns the cleaned version of the sentence
    """
    # remove the character ".", except from floating numbers
    periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
    # remove any "," between digits, eg: 5,6
    commaStrip   = re.compile("(\d)(\,)(\d)")
    # list of punctuations to remove
    punct        = [';', r"/", '[', ']', '"', '{', '}',
                    '(', ')', '=', '+', '\\', '_', '-',
                    '*', ':', '^', '%', '$', '#', '&',
                    '>', '<', '@', '`', ',', '?', '!']
    # contraction mappings
    contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
                    "couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
                    "hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
                    "he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
                    "Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
                    "maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
                    "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
                    "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
                    "she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
                    "somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
                    "somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
                    "someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
                    "something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
                    "there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
                    "they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
                    "wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
                    "whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "where's": "where is", "whereve": "where've", \
                    "whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
                    "whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
                    "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
                    "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
                    "youll": "you'll", "youre": "you're", "youve": "you've"}

    # replace new line with a white space
    inText = sentence.replace('\n', ' ')
    # replace multiple white space with single white space
    inText = inText.replace('\t', ' ')
    inText = inText.strip()
    outText = inText
    for p in punct:
        if (p + ' ' in inText or ' ' + p in inText) or \
           (re.search(commaStrip, inText) != None):
            outText = outText.replace(p, '')
        else:
            outText = outText.replace(p, ' ')
    outText = periodStrip.sub("", outText, re.UNICODE)
    outText = outText.lower().split()
    for wordId, word in enumerate(outText):
        if word in contractions:
            outText[wordId] = contractions[word]
    outText = ' '.join(outText)
    return outText





In [9]:
#read train JSON file
with open(train_json_path, 'r') as f:
    train_data = json.load(f)
f.close()

#read test JSON file
with open(test_json_path, 'r') as f:
    test_data = json.load(f)
f.close()


TOT_QUESTIONS = len(train_data)
indx = list(train_data.keys())
train_indx = indx[:int(TOT_QUESTIONS*DATASET_SPLIT)]
valid_indx = indx[int(TOT_QUESTIONS*DATASET_SPLIT):]





In [119]:

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, list_IDs, image_path, train_input_questions, max_length,to_fit=True,
                 batch_size=128, dim=(49, 512), n_channels=3, n_classes=N_CLASSES, shuffle=True):
        self.list_IDs = list_IDs
        self.train_input_questions = train_input_questions
        self.image_path = image_path
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.to_fit=to_fit
        self.max_length = max_length
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self._generate_X(list_IDs_temp)

        if self.to_fit:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def _generate_X(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        X2 = np.empty((self.batch_size, self.max_length))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self._load_image(self.image_path[i])
            X2[i,] = (self.train_input_questions[i]).tolist()
        ole = [X2, X]
        
        return ole

    def _generate_y(self, list_IDs_temp):
        y = np.empty((self.batch_size, 1), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            y[i] = self.list_IDs[ID]

        return y

#Qua in origine c'era la divisione tra immagine training set o test set ma nel nostro dataset c'è
#un unico insieme di immagini, non so se valga la pena unire togliendo l'if
    def _load_image(self, image_path):
        image = np.load('/content/drive/MyDrive/VQA/features/' + image_path.split('.')[0][-6:] + '.npy')  
        return image

In [112]:
MAX_LEN = 22

def readTrainJson(data, first, last):
        images = []
        questions = []
        answers = []

        #Questo fa schifo è da cambiare
        count = first

        for question in data:
            if count < first:
              continue
            if count >= last:
              break
            q = data[question]
            name = q['image_id']
            quest = process_sentence(q['question'])
            ans = q['answer']
            images.append(name)
            questions.append(quest)
            answers.append(classes[ans])
            count += 1
        return images, questions, answers

def readTestJson(data, first, last):
    quest_id = []
    images = []
    questions = []


    for question in data:
        q = data[question]
        qid = question
        name = q['image_id']
        quest = process_sentence(q['question'])
        quest_id.append(int(qid))
        images.append(name)
        questions.append(quest)
    return images, questions, quest_id

In [120]:
#read train JSON file
with open(train_json_path, 'r') as f:
    train_data = json.load(f)
f.close()

#read test JSON file
with open(test_json_path, 'r') as f:
    test_data = json.load(f)
f.close()

img_h=49
img_w=512
TOT_QUESTIONS = len(train_data)
TRAIN_QUESTIONS = int(TOT_QUESTIONS*DATASET_SPLIT)
VALID_QUESTIONS = TOT_QUESTIONS-TRAIN_QUESTIONS

#extract images, questions and answer (or quest_id) from the train and test files
train_images, train_questions, train_answers = readTrainJson(train_data, 0, TRAIN_QUESTIONS)
valid_images, valid_questions, valid_answers = readTrainJson(train_data, TRAIN_QUESTIONS, TOT_QUESTIONS)
test_images, test_questions, questions_id = readTestJson(test_data, 0, len(test_data))

sequences = tokenizer.texts_to_sequences(train_questions)
train_input_questions = pad_sequences(sequences, maxlen=MAX_LEN)

sequences = tokenizer.texts_to_sequences(valid_questions)
valid_input_questions = pad_sequences(sequences, maxlen=MAX_LEN)


sequences = tokenizer.texts_to_sequences(test_questions)
test_input_questions = pad_sequences(sequences, maxlen=MAX_LEN)
words_number = len(tokenizer.word_index) + 1

training_generator = DataGenerator(train_answers, train_images, train_input_questions, MAX_LEN, batch_size=BATCH_SIZE, dim=(img_h, img_w), n_classes=N_CLASSES)
validation_generator = DataGenerator(valid_answers, valid_images, valid_input_questions, MAX_LEN, batch_size=BATCH_SIZE, dim=(img_h, img_w), n_classes=N_CLASSES)
test_generator = DataGenerator(questions_id, test_images, test_input_questions, MAX_LEN,to_fit=False, batch_size=1, dim=(img_h, img_w), n_classes=N_CLASSES, shuffle=False)

In [121]:
def build_model(max_answers, max_seq_len, vocab_size, dim_d, dim_k, l_rate, d_rate, reg_value):
    """
    Defines the Keras model.

    Arguments
    ----------
    max_answers : Number of output targets of the model.
    max_seq_len : Length of input sequences
    vocab_size  : Size of the vocabulary, i.e. maximum integer index + 1.
    dim_d       : Hidden dimension
    dim_k       : Hidden attention dimension
    l_rate      : Learning rate for the model
    d_rate      : Dropout rate
    reg_value   : Regularization value

    Returns
    ----------
    Returns the Keras model.
    """
    # inputs 
    image_input = Input(shape=(49, 512), name='Image_Input')
    ques_input = Input(shape=(22, ), name='Question_Input')

    # image feature;
    img_model = tf.keras.models.Sequential()
    img_model.add(Flatten())
    img_model.add(Dense(dim_d, activation='relu', \
                    kernel_regularizer=tf.keras.regularizers.l2(reg_value),\
                        kernel_initializer=tf.keras.initializers.glorot_uniform(seed=13)))
    encoded_image = img_model(image_input)
    # question feature;
    word_model = tf.keras.models.Sequential()
    word_model.add(Embedding(input_dim=vocab_size, output_dim=dim_d, input_length=max_seq_len,\
                            mask_zero=True))
    word_model.add(LSTM(units=512, return_sequences=True))
    word_model.add(Dropout(d_rate))
    word_model.add(LSTM(units=512, return_sequences=False))
    word_model.add(Dropout(d_rate))
    word_model.add(Dense(dim_d, activation='relu', \
                    kernel_regularizer=tf.keras.regularizers.l2(reg_value),\
                        kernel_initializer=tf.keras.initializers.glorot_uniform(seed=13)))
    encoded_quest= word_model(ques_input)
    # result
    combine = concatenate([encoded_image,encoded_quest])
    d_layer = Dropout(d_rate)(combine)
    dense1 = Dense(1000,activation='tanh')(d_layer)
    d_layer2 =  Dropout(d_rate)(dense1)
    result = Dense(max_answers, activation='softmax')(d_layer2)

    model = Model(inputs=[ques_input,image_input], outputs=result)

    return model

In [122]:
# params 1


max_seq_len = 22
vocab_size  = len(tokenizer.word_index) + 1
dim_d       = 512
dim_k       = 256
l_rate      = 1e-4
d_rate      = 0.5
reg_value   = 0.01
# create model
model = build_model(N_CLASSES, max_seq_len, vocab_size, dim_d, dim_k, l_rate, d_rate, reg_value)

In [123]:
   # Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy()

# learning rate
lr = 5e-4
#optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
#vqa_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [124]:
# Early Stopping
# --------------
callbacks = []
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)



In [ ]:
model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        epochs=60)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 367 steps, validate for 91 steps
Epoch 1/60
367/367 [==============================] - 98s 266ms/step - loss: 4.7662 - accuracy: 0.5784 - val_loss: 2.6746 - val_accuracy: 0.6383
Epoch 2/60
323/367 [=========================>....] - ETA: 8s - loss: 2.5389 - accuracy: 0.6272

In [ ]:
pred = model.predict_generator(test_generator)

array(['cat', 'cat', 'cat', ..., 'cat', 'cat', 'cat'], dtype='<U11')

In [ ]:

import os
from datetime import datetime
from google.colab import files

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(str(key) + ',' + str(value) + '\n')
    
    return csv_fname

results = {}

for i in range(len(pred)):
    results[test_generator.list_IDs[i]] = np.argmax(pred[i])

name = create_csv(results)

files.download(name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>